In [ ]:
!pip3 install --upgrade pip
!pip3 install selenium
!pip3 install webdriver-manager

In [67]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By


from urllib.request import urlretrieve
from time import sleep

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import os
folder_name = './img_data'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

In [193]:
from tqdm import tqdm

In [2]:
import urllib.request
import json

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/ant671/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ant671/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /Users/ant671/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ant671/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ant671/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 번역

In [159]:
def mt(sentence, client_id, client_secret):
    koText = urllib.parse.quote(sentence)
    data = "source=ko&target=en&text=" + koText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        json_result = json.loads(response_body.decode('utf-8'))
        enText = json_result["message"]["result"]["translatedText"]
        return enText
    else:
        print("Error Code:" + rescode)

## api

In [170]:
client_name = 'client'
with open(f"./{client_name}.json", "r") as file:
    client = json.load(file)
client_id, client_secret = client["client_id"], client["client_secret"]

## 번역  실행

In [171]:
txt_list = []
with open('./golden_summary.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        new_line = line
        txt_list.append(new_line)


In [ ]:
start=0
en_list = []
for line in txt_list[start:]:
    new_line = mt(line.strip(), client_id, client_secret)
    en_list.append(new_line)

In [173]:
print(len(en_list))
print(len(txt_list))

240
35006


- 1차: 0-243
- 2차: 243-373
- 3차: 373-614
- 4차: 614-848
- 5차: 848-1088

## 번역 파일에 기록

In [197]:
file_name = 'en_golden_summary28001_29000.txt'

In [174]:
with open('./'+file_name,'w') as f:
    for line in en_list:
        f.write(line+'\n')

## 번역 파일에서 가져오기

In [1]:
file_name = 'en_golden_21001_22059.txt'

In [3]:
en_txt = []

In [4]:
with open('./'+file_name,'r') as f:
    lines = f.readlines()
    for line in lines:
        en_txt.append(line.strip())

In [5]:
lemmatizer = WordNetLemmatizer()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.append("'m")
stop_words.append("'s")

In [6]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

NameError: name 'webdriver' is not defined

In [7]:
base_URL = 'https://www.google.com/search?q='

In [11]:
proc_sen_list = []
raw_sen_list = []
for txt in en_txt:
    tagged = pos_tag(word_tokenize(txt))
    proc_sen = []
    for t in tagged:
        if t[1].startswith('N') or t[1].startswith('V') or t[1].startswith('J'):
            lemma = lemmatizer.lemmatize(t[0])
            if not lemma in stop_words:
                proc_sen.append(lemma)
    if proc_sen:
        proc_sen_list.append(proc_sen)
        # proc_sen_list.append('+'.join(set(proc_sen)))
        raw_sen_list.append(txt)

In [12]:
print(en_txt[:5])
proc_sen_list[:5]

['When I go to the meeting, I try to take my child and give my husband free time.', "It's the day I decided to do a modeling pack, and my sister said not to worry because she puts it on well.", "We're talking about a song by Mariah Carey that feels like winter.", "Originally, he always watches amazing Saturday (Amazing Saturday), and he is watching Ravi's song because it came out.", "Coffee is popular, so it's a game with a lot of cash."]


[['go', 'meeting', 'try', 'take', 'child', 'give', 'husband', 'free', 'time'],
 ['day', 'decided', 'modeling', 'pack', 'sister', 'said', 'worry', 'put'],
 ["'re", 'talking', 'song', 'Mariah', 'Carey', 'feel', 'winter'],
 ['watch',
  'amazing',
  'Saturday',
  'Amazing',
  'Saturday',
  'watching',
  'Ravi',
  'song',
  'came'],
 ['Coffee', 'popular', 'game', 'lot', 'cash']]

In [226]:
fail_idx = []
for idx in tqdm(range(len(proc_sen_list))):
    search_sen = proc_sen_list[idx]
    file_name = raw_sen_list[idx]
    driver.get(url=base_URL+search_sen+'&tbm=isch')
    result = driver.find_elements(By.XPATH,'//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    try:
        link = result[0].get_attribute('src')
        urlretrieve(link,f'img_data/{file_name}.png')
        driver.implicitly_wait(1)
    except:
        driver.implicitly_wait(30)
        print(idx)
        fail_idx.append(idx)


  5%|▍         | 48/1059 [00:57<21:15,  1.26s/it]

47


  6%|▌         | 61/1059 [01:14<26:38,  1.60s/it]

60


 80%|████████  | 849/1059 [17:07<04:03,  1.16s/it] 

848


100%|██████████| 1059/1059 [21:18<00:00,  1.21s/it]


In [227]:
print(len(proc_sen_list))
print(fail_idx)

1059
[47, 60, 848]


In [229]:
for idx in fail_idx:
    search_sen = proc_sen_list[idx]
    file_name = raw_sen_list[idx]
    driver.get(url=base_URL+search_sen+'&tbm=isch')
    result = driver.find_elements(By.XPATH,'//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    try:
        link = result[0].get_attribute('src')
        urlretrieve(link,f'img_data/{file_name}.png')
        driver.implicitly_wait(1)
    except:
        driver.implicitly_wait(30)
        print(idx)

47
848
